# Reinforcement Learning applied to an adaptation of Splinter

Splinter is an adversarial game, whose objective is to separate the two kings into w different groups, winning the player whose king is inserted in the biggest group.

## Game Analysis

| 0     | 0     | 0     | 0     | 0     |
| ----- | ----- | ----- | ----- | ----- |
| **0** | **w** | **w** | **w** | **0** |
| **0** | **w** | **K** | **w** | **0** |
| **0** | **w** | **K** | **w** | **0** |
| **0** | **w** | **w** | **w** | **0** |
| **0** | **0** | **0** | **0** | **0** |

The parameters of the game such as the horizontal and vertical size and even the gap of the surrounding board can be adjusted.
In this version, there is only one player, and therefore only one color. The objective of the game is to divide the kings into
two different splinters where the one you created is larger.

## Game Adaptation

The game is very complex and computationally heavy to train using q-learning, because of the number of states, so it should be broken into simpler problems.

### First adapation

It is noticeable that the root of the computational problems of this game is in the number of states, which grows exponentially,

In order to reduce the number of possible states all pieces have the same color

In this adaptation, each cell has only 2 possible states:

- Empty cell
- Cell with piece

The objective of this adaptation of the game is to cause a splinter, where the moved piece stays in the biggest group.

### Second Adaptation


### Action space size

For each available cell we have 8 possible moves:

- Move left upwards
- Move upwards
- Move right upwards
- Move left
- Move right
- Move left downwards
- Move downwards
- Move right downwards

The action space size is formulated as

$ ActionSize = 8 H*{size} V*{size} $

| Grid Area | Action Space Size |
| --------- | ----------------- |
| 2         | 16                |
| 4         | 32                |
| 16        | 128               |
| 25        | 200               |
| 36        | 288               |
| 81        | 648               |

### State space size

$ H \gets Horizontal\: Size $,

$ V \gets Vertical\: Size $

$ G \gets Gap $

$ S \gets (H + G \times 2) \times (V + G \times 2),\: Spaces\: in\: board\: grid $

#### Iteration 1 state space size

The state space size can be formulated with the following formula:

$ P \gets H \times V,\: Available\: Pieces $

$ StateSize \gets \sum_{p=0}^{P} \binom{S}{p},\: combinations\: of\: up\: to\: P\: pieces\: on\: S\: spaces $

| Horizontal Size | Vertical Size | Gap | State Space Size     ||
| --------------- | ------------- | --- | -------------------- | -------------------------------------------------------- |
| 2               | 2             | 1   | 2517                 |                                                          |
| 3               | 2             | 1   | 60460                |                                                          |
| 3               | 4             | 1   | 194129627            |                                                          |
| 3               | 4             | 2   | 752157638794         | <--- Original configuration but with same colored pieces |
| 5               | 4             | 1   | 1929894318332        |                                                          |
| 5               | 4             | 2   | 490455609548748676   |                                                          |

#### Iteration 2 state space size

The state space size can be formulated with the following formula:

$ K \gets 2,\: King\: Pieces $

$ P \gets H \times V - K,\: Available\: Pieces $

$ StateSize \gets \sum_{p=0}^{P} [\binom{S}{p} \times \sum_{k=0}^{K} \binom{S-p}{k} ] $

where

$ \binom{S}{p},\: combination\: of\: p\: pieces\: on\: S\: spaces $

$ S - p,\: available\: spaces\: after\: arranging\: the\: p\: pieces $

$ \binom{S-p}{k},\: combinations\: of\: k\: kings\: on\: the\: available\: spaces\: after\: arranging\: the\: p\: pieces $

$ \sum_{k=0}^{K} \binom{S-p}{k},\: total\: combinations\: of\: up\: to\: K\: kings $

$ \binom{S}{p} \times \sum_{k=0}^{K} \binom{S-p}{k},\: total\: combinations\: of\: up\: to\: K\: kings\: for\: every\: combination\: of\: p\: pieces $

| Horizontal Size | Vertical Size | Gap | State Space Size     ||
| --------------- | ------------- | --- | -------------------- | -------------------------------------------------------- |
| 2               | 2             | 1   | 14793                |                                                          |
| 3               | 2             | 1   | 876036               |                                                          |
| 3               | 4             | 1   | 11945875637          |                                                          |
| 3               | 4             | 2   | 49083397468086       | <--- Original configuration but with same colored pieces |
| 5               | 4             | 1   | 326532239723222      |                                                          |
| 5               | 4             | 2   | 91620538908664551100 |                                                          |

The number of states evaluated in this game start to be very large at boards as small as a **3x4 board** with a **gap of 1**.

In [3]:
import sys, os, pathlib
import_path = pathlib.Path(os.path.abspath('')).parent.absolute()
print("Importing classes from", import_path)
os.chdir(import_path)

from model.position import Position

Importing classes from /home/nunoa/Documents/FEUP/THIRD_YEAR/IA/feup-iart-splinter


## Execution
### Iteration 0 Board

In [15]:
from model.board import Board, OriginalBoardBuilder
board: Board = OriginalBoardBuilder(3, 1).build()
print(board)

Horizontal size: 5
Vertical size: 6
------------- Board -------------

[[None, None, None, None, None],
 [None, b   , w   , b   , None],
 [None, w   , B   , w   , None],
 [None, b   , W   , b   , None],
 [None, w   , b   , w   , None],
 [None, None, None, None, None]]


### Iteration 1 Board

In [4]:
from model.board import Board, SimpleBoardBuilder
board: Board = SimpleBoardBuilder(3, 4, 1).build()
print(board)

4
4
4
4
Horizontal size: 5
Vertical size: 6
------------- Board -------------

[[None, None, None, None, None],
 [None, w   , w   , w   , None],
 [None, w   , w   , w   , None],
 [None, w   , w   , w   , None],
 [None, w   , w   , w   , None],
 [None, None, None, None, None]]


### Iteration 2 Board


In [18]:
from model.board import Board, UniColoredBoardBuilder
board: Board = UniColoredBoardBuilder(3, 4, 1).build()
print(board)

Horizontal size: 5
Vertical size: 6
------------- Board -------------

[[None, None, None, None, None],
 [None, w   , w   , w   , None],
 [None, w   , W   , w   , None],
 [None, w   , W   , w   , None],
 [None, w   , w   , w   , None],
 [None, None, None, None, None]]


### Iteration 2 Setup

In [20]:
import gym
from controller.game_controller import GameController, SimpleRewardSystem, UniColoredGameController
from controller.reward_system import BiggestDiffUniColoredRewardSystem
from model.board import OriginalBoardBuilder, Board, UniColoredBoardBuilder
from model.gym import SplinterEnv
from model.position import Position
from model.state import StateFactory


move_factory = StateFactory()
board: Board = UniColoredBoardBuilder(3, 4, 1).build()
reward_system: SimpleRewardSystem = BiggestDiffUniColoredRewardSystem()
game_controller: GameController = UniColoredGameController(
    board, True, reward_system)

env: gym.Env = SplinterEnv(board, game_controller, move_factory)

### Q-Learning

In [25]:
from abc import ABC, abstractmethod
import random
import gym
import numpy as np
from model.board import Board
from model.gym import SplinterEnv
from model.position import Position, Transformation
from model.qtable import QTable
from model.state import StateFactory

class Learner(ABC):
    def __init__(self, env: gym.Env, board: Board, action_factory: StateFactory) -> None:
        super().__init__()

        self.board = board
        self.env = env
        self.action_factory = action_factory
        self._board_dict = {}
        self._max_dict = 0

        grid_size = board.horizontal_size * board.vertical_size
        action_size = action_factory.state_size() * grid_size

        self._q_table = QTable(action_size)

    def is_invalid_move(self, action):
        pure_action = self.parse_action(action)
        result = not self.env.game_controller.check_valid_move(pure_action)[
            0]
        return result

    def choose_method(self, state, epsilon):
        exp_exp_tradeoff = random.uniform(0, 1)

        if exp_exp_tradeoff > epsilon:
            # Always choose a valid action
            actions_scores = self._q_table.get_state(state)
            invalid_moves = [x for x in range(
                actions_scores.size) if self.is_invalid_move(x)]
            mask = np.zeros(actions_scores.size, dtype=bool)
            mask[invalid_moves] = True
            return np.argmax(
                np.ma.array(actions_scores, mask=mask))
        
        return self.env.action_space.sample()

    def parse_state(self, state: Board) -> int:
        key = str(state.board)
        result = self._board_dict.get(key)

        if (result == None):
            self._board_dict.update({key: self._max_dict})
            result = self._max_dict
            self._max_dict += 1
        return result

    def parse_action(self, action: int) -> Transformation:
        actions_size = self.action_factory.state_size()
        piece_idx = action // actions_size
        x, y = piece_idx % self.board.horizontal_size, piece_idx // self.board.horizontal_size
        move_idx = action % actions_size

        position = Position(x, y)
        move = self.action_factory.build(move_idx)

        return move.get_move(position)

    @abstractmethod
    def train(self):
        pass

    def run(self):
        for episode in range(5):
            pure_state = self.env.reset()
            state = self.parse_state(pure_state)

            step = 0
            done = False
            print("\n\n\n****************************************************")
            print("****************************************************")
            print("****************************************************")
            print("EPISODE ", episode + 1)

            episode_reward = 0

            for step in range(20):
                self.env.render()
                action = self.choose_method(state, 0)
                pure_action = self.parse_action(action)
                print("ACTION TAKEN:", pure_action)

                new_pure_state, reward, done, info = self.env.step(
                    pure_action, 0)

                episode_reward += reward

                if done:
                    break
                state = self.parse_state(new_pure_state)
            print("\nEPISODE REWARD", episode_reward)
            print("STEPS TAKEN", step + 1)

            self.env.render()
        self.env.close()


class QLearner(Learner):
    def __init__(self,  env: SplinterEnv, board: Board, action_factory: StateFactory) -> None:
        super().__init__(env, board, action_factory)

    def train(self):
        total_episodes = 10000    # Total episodes
        max_steps = 20                # Max steps per episode

        learning_rate = 1           # Learning rate
        gamma = 0.95                  # Discounting rate

        # Exploration parameters
        epsilon = 1.0                 # Exploration rate
        max_epsilon = 1.0             # Exploration probability at start
        min_epsilon = 0.01            # Minimum exploration probability
        decay_rate = 0.0001

        # For life or until learning is stopped
        for episode in range(total_episodes):
            print("Episode:", episode+1)
            # Reset the environment
            pure_state = self.env.reset()
            state = self.parse_state(pure_state)

            step = 0
            player = 0
            done = False

            for step in range(max_steps):
                action = self.choose_method(state, epsilon)                

                pure_action = self.parse_action(action)

                new_pure_state, reward, done, info = self.env.step(
                    pure_action, player)

                new_state = self.parse_state(new_pure_state)

                best_new_action = np.max(self._q_table.get_state(new_state))
                current_action = self._q_table.get(state, action)
                new_value = current_action + learning_rate * (
                    reward + gamma * best_new_action - current_action)

                self._q_table.update(state, action, new_value)

                # Our new state is state
                state = new_state

                if done == True:
                    break

            # reduce epsilon (because we need less and less exploration)
            epsilon = min_epsilon + \
                (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
            print(epsilon)

        print(self._q_table.get_q_table())

        self.env.reset()

In [26]:

learner: Learner = QLearner(env, board, move_factory)

learner.train()
learner.run()

Episode: 1
1.0
Episode: 2
0.999901004949835
Episode: 3
0.9998020197986801
Episode: 4
0.9997030445455454
Episode: 5
0.999604079189441
Episode: 6
0.9995051237293776
Episode: 7
0.9994061781643654
Episode: 8
0.9993072424934148
Episode: 9
0.9992083167155369
Episode: 10
0.9991094008297421
Episode: 11
0.9990104948350412
Episode: 12
0.9989115987304453
Episode: 13
0.9988127125149655
Episode: 14
0.9987138361876128
Episode: 15
0.9986149697473984
Episode: 16
0.9985161131933338
Episode: 17
0.9984172665244303
Episode: 18
0.9983184297396994
Episode: 19
0.9982196028381529
Episode: 20
0.9981207858188024
Episode: 21
0.9980219786806598
Episode: 22
0.9979231814227368
Episode: 23
0.9978243940440459
Episode: 24
0.9977256165435988
Episode: 25
0.997626848920408
Episode: 26
0.9975280911734855
Episode: 27
0.9974293433018441
Episode: 28
0.997330605304496
Episode: 29
0.997231877180454
Episode: 30
0.9971331589287308
Episode: 31
0.9970344505483393
Episode: 32
0.9969357520382922
Episode: 33
0.9968370633976026
Episod

### SARSA

In [31]:
class SARSALearner(Learner):
    def __init__(self,  env: SplinterEnv, board: Board, action_factory: StateFactory) -> None:
        super().__init__(env, board, action_factory)

    def train(self):
        total_episodes = 60000    # Total episodes
        max_steps = 20                # Max steps per episode

        learning_rate = 1           # Learning rate
        gamma = 0.95                  # Discounting rate

        # Exploration parameters
        epsilon = 1.0                 # Exploration rate
        max_epsilon = 1.0             # Exploration probability at start
        min_epsilon = 0.01            # Minimum exploration probability
        decay_rate = 0.000035

        # For life or until learning is stopped
        for episode in range(total_episodes):
            print("Episode:", episode+1)
            # Reset the environment
            pure_state = self.env.reset()
            state = self.parse_state(pure_state)
            action1 = self.choose_method(state, epsilon)

            step = 0
            player = 0
            done = False

            for step in range(max_steps):
                pure_action = self.parse_action(action1)

                new_pure_state, reward, done, info = self.env.step(
                    pure_action, player)

                new_state = self.parse_state(new_pure_state)

                action2 = self.choose_method(new_state, epsilon)

                best_new_action = self._q_table.get(new_state, action2)
                current_action = self._q_table.get(state, action1)
                new_value = current_action + learning_rate * (
                    reward + gamma * best_new_action - current_action)

                self._q_table.update(state, action1, new_value)

                # Our new state is state
                state = new_state
                action1 = action2

                if done == True:
                    break

            # reduce epsilon (because we need less and less exploration)
            epsilon = min_epsilon + \
                (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
            print(epsilon)

        print(self._q_table.get_q_table())

        self.env.reset()


In [32]:
learner: Learner = SARSALearner(env, board, move_factory)

learner.train()
learner.run()

Episode: 1
1.0
Episode: 2
0.9999653506063679
Episode: 3
0.9999307024254434
Episode: 4
0.999896055457184
Episode: 5
0.9998614097015472
Episode: 6
0.9998267651584908
Episode: 7
0.9997921218279721
Episode: 8
0.9997574797099487
Episode: 9
0.9997228388043782
Episode: 10
0.9996881991112182
Episode: 11
0.9996535606304263
Episode: 12
0.9996189233619599
Episode: 13
0.9995842873057768
Episode: 14
0.9995496524618344
Episode: 15
0.9995150188300902
Episode: 16
0.9994803864105021
Episode: 17
0.9994457552030274
Episode: 18
0.9994111252076238
Episode: 19
0.9993764964242488
Episode: 20
0.99934186885286
Episode: 21
0.9993072424934148
Episode: 22
0.9992726173458713
Episode: 23
0.9992379934101865
Episode: 24
0.9992033706863184
Episode: 25
0.9991687491742244
Episode: 26
0.9991341288738621
Episode: 27
0.9990995097851891
Episode: 28
0.9990648919081629
Episode: 29
0.9990302752427414
Episode: 30
0.9989956597888818
Episode: 31
0.9989610455465421
Episode: 32
0.9989264325156795
Episode: 33
0.9988918206962517
Epis